!pip uninstall -y scopyon
!pip install git+https://github.com/ecell/scopyon
!pip freeze | grep scopyon

In [1]:
import mlflow
mlflow.start_run(run_name="generation", nested=True)

<ActiveRun: >

In [2]:
seed = 123
num_samples = 5
exposure_time = 33.0e-3
interval = 33.0e-3
num_frames = 10
Nm = [100, 100, 100]
Dm = [0.222e-12, 0.032e-12, 0.008e-12]
transmat = [
    [0.0, 0.5, 0.0],
    [0.5, 0.0, 0.2],
    [0.0, 1.0, 0.0]]

/home/azureuser/miniconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
from mlflow import log_metric, log_param, log_artifacts
log_param("seed", seed)
log_param("num_samples", num_samples)
log_param("exposure_time", exposure_time)
log_param("interval", interval)
log_param("num_frames", num_frames)

In [4]:
nproc = 8

In [5]:
# !pip install mlflow

In [6]:
import numpy
rng = numpy.random.RandomState(seed)

In [7]:
import scopyon

In [8]:
config = scopyon.DefaultConfiguration()
config.default.effects.photo_bleaching.switch = False
config.default.detector.exposure_time = exposure_time
pixel_length = config.default.detector.pixel_length / config.default.magnification
L_2 = config.default.detector.image_size[0] * pixel_length * 0.5
L_2

/home/azureuser/miniconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


1.6995850622406638e-05

In [9]:
config.environ.processes = nproc

In [10]:
timepoints = numpy.linspace(0, interval * num_frames, num_frames + 1)
ndim = 2

In [11]:
import pathlib
artifacts = pathlib.Path("./artifacts")
artifacts.mkdir(parents=True, exist_ok=True)

In [12]:
config.save(artifacts / 'config.yaml')

In [13]:
for i in range(num_samples):
    samples = scopyon.sample(timepoints, N=Nm, lower=-L_2, upper=+L_2, ndim=ndim, D=Dm, transmat=transmat, rng=rng)
    inputs = [(t, numpy.hstack((points[:, : ndim], points[:, [ndim + 1]], numpy.ones((points.shape[0], 1), dtype=numpy.float64)))) for t, points in zip(timepoints, samples)]
    ret = list(scopyon.generate_images(inputs, num_frames=num_frames, config=config, rng=rng, full_output=True))
    
    inputs_ = []
    for t, data in inputs:
        inputs_.extend(([t] + list(row) for row in data))
    inputs_ = numpy.array(inputs_)
    numpy.save(artifacts / f"inputs{i:03d}.npy", inputs_)

    numpy.save(artifacts / f"images{i:03d}.npy", numpy.array([img.as_array() for img, infodict in ret]))

    true_data = []
    for t, (_, infodict) in zip(timepoints, ret):
        true_data.extend([t, key] + list(value) for key, value in infodict['true_data'].items())
    true_data = numpy.array(true_data)
    numpy.save(artifacts / f"true_data{i:03d}.npy", true_data)

/home/azureuser/miniconda3/lib/python3.8/asyncio/events.py:81: DeprecationWarning: `run_cell_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  self._context.run(self._callback, *self._args)


In [15]:
!ls ./artifacts

config.yaml    images003.npy  inputs002.npy	true_data001.npy
images000.npy  images004.npy  inputs003.npy	true_data002.npy
images001.npy  inputs000.npy  inputs004.npy	true_data003.npy
images002.npy  inputs001.npy  true_data000.npy	true_data004.npy


In [14]:
log_artifacts("./artifacts")
mlflow.end_run()